### Installations/setup

In [5]:
# Install libs (run once)
!pip install -q lightgbm prophet scikit-learn joblib pandas geopandas matplotlib seaborn tensorflow

# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
import os

### Get Data

In [6]:
import pandas as pd
import numpy as np
from pathlib import Path

# Path
DATA_PATH = "/content/drive/MyDrive/Colab Notebooks/PHC DataThon Project/NGGC81FL.csv"
OUT_PATH = "/content/drive/MyDrive/Colab Notebooks/PHC DataThon Project/site_year_2000_2020.csv"

df = pd.read_csv(DATA_PATH)

df.head()

,DHSID,GPS_Dataset,DHSCC,DHSYEAR,DHSCLUST,SurveyID,All_Population_Count_2000,All_Population_Count_2005,All_Population_Count_2010,All_Population_Count_2015,...,UN_Population_Density_2000,UN_Population_Density_2005,UN_Population_Density_2010,UN_Population_Density_2015,UN_Population_Density_2020,Wet_Days_2000,Wet_Days_2005,Wet_Days_2010,Wet_Days_2015,Wet_Days_2020
0,NG202100000001,NGGE81FL,NG,2021,1,NG2021MIS,4383.732422,10511.250980,15821.37695,12767.423830,...,122.340126,137.952301,156.200867,176.804611,198.476639,7.454952,7.298984,7.694158,7.095560,7.626615
1,NG202100000002,NGGE81FL,NG,2021,2,NG2021MIS,10095.720700,13841.389650,15144.12500,17101.576170,...,328.920227,370.894653,419.957245,475.351868,533.618652,7.029486,6.815495,7.203261,6.596217,7.128038
2,NG202100000003,NGGE81FL,NG,2021,3,NG2021MIS,34067.722660,44976.214840,55211.96875,55673.691410,...,175.446625,197.835861,224.005936,253.553528,284.633148,8.311914,8.184244,8.568884,8.013810,8.531879
3,NG202100000004,NGGE81FL,NG,2021,4,NG2021MIS,4087.095947,5683.123535,6247.22168,9392.799805,...,155.192780,174.997360,198.146332,224.282898,251.774643,8.987461,8.872630,9.269448,8.747023,9.237495
4,NG202100000005,NGGE81FL,NG,2021,5,NG2021MIS,543.357239,545.644714,962.36676,1112.796753,...,62.213058,70.152237,79.432098,89.909622,100.930397,6.834844,6.670360,6.903941,6.319201,6.890534


In [7]:
# Inspect columns and group by base variable names (strip year suffix)
cols = df.columns.tolist()

# Identify columns with suffixes _2000, _2005, _2010, _2015, _2020
years = ['2000','2005','2010','2015','2020']

# build dictionary: base_name -> [col_2000, col_2005, ...]
from collections import defaultdict
groups = defaultdict(list)
meta_cols = []  # columns not part of time-series (IDs, DHSYEAR, etc)

for c in cols:
    matched = False
    for y in years:
        if c.endswith('_' + y):
            base = c[:-(len(y)+1)]
            groups[base].append((y,c))
            matched = True
            break
    if not matched:
        # preserve key identifiers
        meta_cols.append(c)

# Melt / long-format building
records = []
for idx, row in df.iterrows():
    # get id/metadata
    meta = {k: row[k] for k in meta_cols}
    site_id = row.get('DHSID', f"site_{idx}")
    for base, pairs in groups.items():
        local = {'DHSID': site_id}
        for k in ['GPS_Dataset','DHSCC','DHSYEAR','DHSCLUST','SurveyID']:
            if k in row:
                local[k] = row[k]
        # for each year value add a record
        for y, colname in pairs:
            rec = local.copy()
            rec['year'] = int(y)
            rec['variable'] = base
            rec['value'] = row[colname] if not pd.isnull(row[colname]) else np.nan
            records.append(rec)

# Create a long table with rows per site-year-variable
long = pd.DataFrame(records)

# Pivot so each row = site + year with columns for each base variable
pivot = long.pivot_table(index=['DHSID','GPS_Dataset','DHSCC','DHSCLUST','SurveyID','year'],
                         columns='variable', values='value').reset_index()

# Ensure numeric columns and sort
pivot = pivot.sort_values(['DHSID','year']).reset_index(drop=True)

pivot

variable,DHSID,GPS_Dataset,DHSCC,DHSCLUST,SurveyID,year,All_Population_Count,Aridity,Day_Land_Surface_Temp,Diurnal_Temperature_Range,...,Mean_Temperature,Minimum_Temperature,Night_Land_Surface_Temp,PET,Precipitation,Rainfall,U5_Population,UN_Population_Count,UN_Population_Density,Wet_Days
0,NG202100000001,NGGE81FL,NG,1,NG2021MIS,2000,4383.732422,31.704765,30.638594,11.171659,...,27.257683,21.690279,20.707235,3.938889,124.881561,1416.583618,684.813354,2609.124268,122.340126,7.454952
1,NG202100000001,NGGE81FL,NG,1,NG2021MIS,2005,10511.250980,27.991472,30.364435,10.308854,...,27.836935,22.707378,21.457466,3.842708,107.563065,1140.103882,1960.725586,2942.082031,137.952301,7.298984
2,NG202100000001,NGGE81FL,NG,1,NG2021MIS,2010,15821.376950,29.045416,31.600908,10.533421,...,28.188629,22.945486,21.280714,4.058941,117.893623,1297.106934,2477.673584,3331.265625,156.200867,7.694158
3,NG202100000001,NGGE81FL,NG,1,NG2021MIS,2015,12767.423830,26.011143,31.521997,10.735026,...,27.742622,22.397917,21.498293,4.054644,105.465927,1158.834351,2193.243896,3770.677734,176.804611,7.095560
4,NG202100000001,NGGE81FL,NG,1,NG2021MIS,2020,17068.992190,31.539787,33.144665,10.661372,...,27.659506,22.351433,21.983686,3.900391,123.017494,1183.953491,2805.607178,4232.873047,198.476639,7.626615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830,NG202100000568,NGGE81FL,NG,568,NG2021MIS,2000,4261.649414,29.706898,27.699736,10.146904,...,27.050636,22.007364,18.397335,3.404644,101.141426,1199.603882,572.356995,8147.553711,382.070435,8.662888
2831,NG202100000568,NGGE81FL,NG,568,NG2021MIS,2005,3669.351563,28.342777,27.674568,9.051606,...,27.347136,22.851114,19.556574,3.251201,92.148064,1090.408325,549.437134,9326.541992,437.357758,8.957118
2832,NG202100000568,NGGE81FL,NG,568,NG2021MIS,2010,5887.438477,37.519463,27.474140,9.338700,...,27.633970,22.983116,20.072605,3.312978,124.301132,1452.943848,617.235779,10720.339840,502.718353,10.646020
2833,NG202100000568,NGGE81FL,NG,568,NG2021MIS,2015,6615.965332,25.763845,28.961945,9.380773,...,27.480093,22.807337,20.305695,3.447917,88.831596,1239.777100,909.266357,12318.337890,577.654663,9.085263


In [8]:
# Save data
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)
pivot.to_csv(OUT_PATH, index=False)
print("Saved processed file with shape:", pivot.shape)

Saved processed file with shape: (2835, 27)


### Interpolate to fill in missing year values

The interpolation method that will be adopted for this is  PCHIP (Piecewise Cubic Hermite Interpolating Polynomial). because of the time-series nature of the data and the relationship between variables like precipitation, vegetation index, and malaria prevalence is non-linear.

PHICP has high accuracy with stability and no overshooting, and it is ideal for sparse evironmental data such as this.

In [9]:
def interpolate_pchip(df, start=2000, end=2020):
    out_rows = []
    for dhsid, g in df.groupby('DHSID'):
        g2 = g.set_index('year').sort_index()
        years_full = pd.RangeIndex(start, end + 1)
        g2 = g2.reindex(years_full)

        for col in g2.columns:
            if col in ['DHSID', 'GPS_Dataset', 'DHSCC', 'DHSCLUST', 'SurveyID']:
                g2[col] = g2[col].ffill().bfill()
            else:
                try:
                    g2[col] = pd.to_numeric(g2[col], errors='coerce')
                    g2[col] = g2[col].interpolate(method='pchip', limit_direction='both')
                except Exception:
                    pass

        g2 = g2.reset_index().rename(columns={'index': 'year'})
        g2['DHSID'] = dhsid
        out_rows.append(g2)

    return pd.concat(out_rows, ignore_index=True)

# Use the stable version
site_year_pchip = interpolate_pchip(pivot, 2000, 2020)

site_year_pchip.head(20)

variable,year,DHSID,GPS_Dataset,DHSCC,DHSCLUST,SurveyID,All_Population_Count,Aridity,Day_Land_Surface_Temp,Diurnal_Temperature_Range,...,Mean_Temperature,Minimum_Temperature,Night_Land_Surface_Temp,PET,Precipitation,Rainfall,U5_Population,UN_Population_Count,UN_Population_Density,Wet_Days
0,2000,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,4383.732422,31.704765,30.638594,11.171659,...,27.257683,21.690279,20.707235,3.938889,124.881561,1416.583618,684.813354,2609.124268,122.340126,7.454952
1,2001,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,5675.562182,30.538178,30.504804,10.901896,...,27.392628,21.963753,20.940616,3.896581,119.094131,1324.697421,1005.853567,2671.287798,125.254934,7.383495
2,2002,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,6935.630878,29.519731,30.423653,10.665417,...,27.519361,22.213803,21.146093,3.868690,114.300855,1248.238908,1301.675839,2735.770405,128.278482,7.337910
3,2003,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,8162.536551,28.713248,30.381981,10.477538,...,27.636546,22.428826,21.309902,3.852335,110.669445,1190.075521,1564.569227,2802.467023,131.405843,7.312457
4,2004,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,9354.877239,28.182554,30.366628,10.353578,...,27.742849,22.597219,21.418281,3.844635,108.367611,1153.074698,1786.822790,2871.272587,134.632092,7.301395
5,2005,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,10511.250980,27.991472,30.364435,10.308854,...,27.836935,22.707378,21.457466,3.842708,107.563065,1140.103882,1960.725586,2942.082031,137.952301,7.298984
6,2006,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,11791.774767,28.101082,30.493028,10.325410,...,27.929532,22.781534,21.439084,3.865197,108.637443,1156.432199,2108.668721,3015.285182,141.384749,7.340082
7,2007,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,13199.719843,28.362460,30.799674,10.367505,...,28.023755,22.846759,21.395249,3.918822,111.199421,1195.368956,2248.644392,3091.127193,144.940932,7.438085
8,2008,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,14498.415623,28.674428,31.165670,10.423778,...,28.106848,22.898717,21.342931,3.982827,114.257267,1241.841860,2366.343296,3169.285988,148.605747,7.555057
9,2009,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,15451.191521,28.935806,31.472315,10.482870,...,28.166058,22.933071,21.299096,4.036453,116.819245,1280.778617,2447.456128,3249.439491,152.364093,7.653060


In [10]:
# Save data
Path(OUT_PATH).parent.mkdir(parents=True, exist_ok=True)
site_year_pchip.to_csv(OUT_PATH, index=False)
print("Saved processed file with shape:", site_year_pchip.shape)

Saved processed file with shape: (11907, 27)


### Feature Engineering

In [11]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

site_year = pd.read_csv(OUT_PATH)

# Ensure the key label column exists
if 'Malaria_Prevalence' not in site_year.columns:
    # if interpolation left separate columns or other naming issues, adapt accordingly
    pass

# Sort features
site_year = site_year.sort_values(['DHSID','year'])

site_year.head()

,year,DHSID,GPS_Dataset,DHSCC,DHSCLUST,SurveyID,All_Population_Count,Aridity,Day_Land_Surface_Temp,Diurnal_Temperature_Range,...,Mean_Temperature,Minimum_Temperature,Night_Land_Surface_Temp,PET,Precipitation,Rainfall,U5_Population,UN_Population_Count,UN_Population_Density,Wet_Days
0,2000,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,4383.732422,31.704765,30.638594,11.171659,...,27.257683,21.690279,20.707235,3.938889,124.881561,1416.583618,684.813354,2609.124268,122.340126,7.454952
1,2001,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,5675.562182,30.538178,30.504804,10.901896,...,27.392628,21.963753,20.940616,3.896581,119.094131,1324.697421,1005.853567,2671.287798,125.254934,7.383495
2,2002,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,6935.630878,29.519731,30.423653,10.665417,...,27.519361,22.213803,21.146093,3.868690,114.300855,1248.238908,1301.675839,2735.770405,128.278482,7.337910
3,2003,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,8162.536551,28.713248,30.381981,10.477538,...,27.636546,22.428826,21.309902,3.852335,110.669445,1190.075521,1564.569227,2802.467023,131.405843,7.312457
4,2004,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,9354.877239,28.182554,30.366628,10.353578,...,27.742849,22.597219,21.418281,3.844635,108.367611,1153.074698,1786.822790,2871.272587,134.632092,7.301395


In [12]:
import warnings
warnings.filterwarnings("ignore")

# create 1-year lag of prevalence (group by site)
site_year['prev_lag1'] = site_year.groupby('DHSID')['Malaria_Prevalence'].shift(1)
# Add rolling mean feature
site_year['prev_roll3'] = site_year.groupby('DHSID')['Malaria_Prevalence'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True)

# fill nans with forward and backward fill
site_year.fillna(method='ffill', inplace=True)
site_year.fillna(method='bfill', inplace=True)

# feature list (drop identifiers and label)
features = [c for c in site_year.columns if c not in ['DHSID','GPS_Dataset','DHSCC','DHSCLUST','SurveyID','year','Malaria_Prevalence']]

# scale
scaler = StandardScaler()
site_year[features] = scaler.fit_transform(site_year[features])

# Save feature list and scaler for use later
import joblib
joblib.dump(scaler, "/content/drive/MyDrive/Colab Notebooks/PHC DataThon Project/scaler_site_year.joblib")

print("Feature list saved successfully.")

Feature list saved successfully.


### Modelling Strategy/ Train-Test-Split

In [13]:
site_year.to_csv("/content/drive/MyDrive/Colab Notebooks/PHC DataThon Project/model_ready_data.csv", index=False)

In [14]:
train_df = site_year[site_year['year'] <= 2015].copy()
test_df = site_year[site_year['year'] == 2020].copy()

X_train = train_df[features].values
y_train = train_df['Malaria_Prevalence'].values

X_test = test_df[features].values
y_test = test_df['Malaria_Prevalence'].values

print("Split Successful")

Split Successful


In [20]:
X_train[0:5]

array([[-0.36773658,  0.02620894, -0.27494785,  0.16311043,  0.09251787,
         0.        , -0.5633149 , -0.18641356,  0.79740222,  0.04525987,
        -0.11282646, -0.24141621,  0.2033339 , -0.20439351,  0.29557906,
        -0.01644265, -0.3994183 , -0.30761768, -0.30964626, -0.28344082,
         0.55074829,  0.48334025],
       [-0.34574502, -0.0391985 , -0.31860103,  0.03813641,  0.10864784,
         0.        , -0.66604504, -0.17021124,  0.92783511,  0.05073947,
         0.04222706, -0.00594656,  0.39816831, -0.24118146,  0.15924821,
        -0.15564067, -0.35871001, -0.30634798, -0.30841155, -0.30882114,
         0.55074829,  0.55184012],
       [-0.32429415, -0.09630012, -0.34507918, -0.07141842,  0.12365793,
         0.        , -0.75353296, -0.14564028,  1.02274293,  0.06558946,
         0.18784449,  0.20935391,  0.56970773, -0.26543323,  0.04633605,
        -0.27146735, -0.32119937, -0.30503091, -0.30713078, -0.32501192,
         0.68910247,  0.60662408],
       [-0.30340783

In [21]:
X_test[0:5]

array([[-0.15178813,  0.01695906,  0.54274031, -0.0732922 , -0.0463977 ,
         0.        ,  2.72419596,  0.9905534 , -0.48318373,  0.14077399,
         0.34887305,  0.32785704,  1.26896126, -0.23786862,  0.25166841,
        -0.36885306, -0.13049901, -0.27445229, -0.27739494, -0.22246982,
        -0.59293163, -0.63882082],
       [-0.09791856, -0.10143534,  0.30300308, -0.04699196, -0.67090589,
         0.        ,  2.62635369,  1.16503696, -0.92287263,  0.24784398,
         0.50742632,  0.42459909,  2.35049432, -0.16271661,  0.11756684,
        -0.28589754, -0.10811419, -0.16280421, -0.13542925, -0.39955417,
        -0.94683887, -0.98968807],
       [ 0.68542603,  0.25849793,  0.07754657, -0.2766185 ,  0.07747363,
         0.        ,  1.54993604,  0.70574488, -0.74450308, -0.16005079,
         0.04506277,  0.29661422,  1.69507812, -0.38875145,  0.50095287,
        -0.38974964,  0.9625447 , -0.23681706, -0.24089916,  0.09906196,
        -0.7629594 , -0.81062484],
       [-0.27481038

In [22]:
y_train[0:5]

array([0.44374207, 0.46789608, 0.48479546, 0.49552426, 0.50116656])

In [23]:
y_test[0:5]

array([0.25795457, 0.19559829, 0.22364542, 0.42407712, 0.30223989])

### Model Test Exploration

Three deep-learning architectures would be tried on snippet of the data and the best performing model will be used for the full data.

The three architectures chose will be architectures that will work soundly given the interpolated annual series and pooled data across many sites. Here are the three options

**Model A — Dense Feedforward Neural Network (MLP)**

- This will treat each (site, year) row as independent input (features include current-year covariates and lag features).
- MLP models are very simple, fast, and robust with tabular data. This will be a good baseline choice.

**Model B — LSTM Sequence Model**

- For each site, this will feed sequences of past T years (e.g., 10 years 2005–2014) to predict next-year prevalence.
- LSTM models have temporal patterns but they need sequential data.

**Model C — Transformer-like Time-Series Model (Temporal Transformer Encoder)**

-These are similar to LSTM but uses self-attention.
- They are good at learning long-range dependencies.
- They are also strong for many time steps and many sites.
- They are more complex but powerful.

The Keras implementations of all three will be run and comparism will be done on the MAE on test data for(2020). Then the best will be chosen for full retrain.

**Model A (MLP Model)**

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

def build_mlp(input_dim, hidden_units=[256,128], dropout=0.2):
    inp = layers.Input(shape=(input_dim,))
    x = inp
    for h in hidden_units:
        x = layers.Dense(h, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropout)(x)
    out = layers.Dense(1, activation='linear')(x)
    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

mlp = build_mlp(X_train.shape[1])
es = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = mlp.fit(X_train, y_train, validation_split=0.1, epochs=200, batch_size=128, callbacks=[es])
preds_mlp = mlp.predict(X_test).squeeze()

Epoch 1/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 1.2491 - mae: 0.8466 - val_loss: 0.0668 - val_mae: 0.2349
Epoch 2/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.4538 - mae: 0.5080 - val_loss: 0.0589 - val_mae: 0.2065
Epoch 3/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.3178 - mae: 0.4162 - val_loss: 0.0253 - val_mae: 0.1379
Epoch 4/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 0.2091 - mae: 0.3413 - val_loss: 0.0192 - val_mae: 0.1150
Epoch 5/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.1487 - mae: 0.2869 - val_loss: 0.0185 - val_mae: 0.1082
Epoch 6/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.1132 - mae: 0.2475 - val_loss: 0.0108 - val_mae: 0.0710
Epoch 7/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0836 - mae: 0.2123 - val_loss: 0.0070 - val_mae: 0.0695
Epoch 8/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0574 - mae: 0.1776 - val_loss: 0.0053 - val_mae: 0.0552
Epoch 9/200
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss:

**Model B (LSTM Model)**

In [25]:
import numpy as np

# Build sequences across all sites
seq_len = 10

def make_sequences(df, feature_cols, label_col='Malaria_Prevalence', seq_len=10):
    Xs, ys = [], []
    for site, g in df.groupby('DHSID'):
        g = g.sort_values('year')
        arr = g[feature_cols].values
        labels = g[label_col].values
        if len(arr) <= seq_len:
            continue
        for i in range(seq_len, len(arr)):
            Xs.append(arr[i-seq_len:i])
            ys.append(labels[i])
    return np.array(Xs), np.array(ys)

X_seq_train, y_seq_train = make_sequences(train_df, features, seq_len=10)
X_seq_test, y_seq_test = make_sequences(test_df, features, seq_len=10)  # note: test may have too short sequences; we handle differently below

# Define LSTM model
def build_lstm(input_shape, units=64):
    inp = layers.Input(shape=input_shape)  # (seq_len, n_features)
    x = layers.LSTM(units, return_sequences=False)(inp)
    x = layers.Dense(32, activation='relu')(x)
    out = layers.Dense(1)(x)
    model = models.Model(inp, out)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

lstm = build_lstm((seq_len, X_seq_train.shape[2]))
es = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lstm.fit(X_seq_train, y_seq_train, validation_split=0.1, epochs=200, batch_size=64, callbacks=[es])
# For test: you need sequences ending at year 2019 to predict 2020; ensure you build test sequences accordingly

Epoch 1/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.0720 - mae: 0.1931 - val_loss: 0.0093 - val_mae: 0.0728
Epoch 2/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0038 - mae: 0.0482 - val_loss: 0.0073 - val_mae: 0.0658
Epoch 3/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0021 - mae: 0.0355 - val_loss: 0.0053 - val_mae: 0.0560
Epoch 4/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0014 - mae: 0.0293 - val_loss: 0.0055 - val_mae: 0.0563
Epoch 5/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0012 - mae: 0.0272 - val_loss: 0.0044 - val_mae: 0.0505
Epoch 6/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 9.8917e-04 - mae: 0.0248 - val_loss: 0.0043 - val_mae: 0.0492
Epoch 7/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 7.7191e-04 - mae: 0.0216 - val_loss: 0.0040 - val_mae: 0.0468
Epoch 8/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 7.1629e-04 - mae: 0.0210 - val_loss: 0.0038 - val_mae: 0.0458
Epoch 9/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16

**Model C (Transformer Model)**

In [26]:
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention

def build_transformer(seq_len, n_features, d_model=64, n_heads=4, ff_dim=128):
    inp = layers.Input(shape=(seq_len, n_features))
    # Project input to d_model
    x = layers.Dense(d_model)(inp)
    # Positional encoding: use simple sin/cos or learned embedding
    pos = tf.range(start=0, limit=seq_len, delta=1)
    pos_emb = layers.Embedding(input_dim=seq_len, output_dim=d_model)(pos)
    x = x + pos_emb
    # Encoder block
    attn_output = MultiHeadAttention(num_heads=n_heads, key_dim=d_model//n_heads)(x, x)
    x = LayerNormalization()(x + attn_output)
    ff = layers.Dense(ff_dim, activation='relu')(x)
    ff = layers.Dense(d_model)(ff)
    x = LayerNormalization()(x + ff)
    # Pooling and head
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    out = layers.Dense(1)(x)
    model = models.Model(inputs=inp, outputs=out)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

transformer = build_transformer(seq_len, X_seq_train.shape[2])
transformer.fit(X_seq_train, y_seq_train, validation_split=0.1, epochs=200, batch_size=64, callbacks=[es])

Epoch 1/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.5696 - mae: 0.4923 - val_loss: 0.0200 - val_mae: 0.1105
Epoch 2/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0106 - mae: 0.0803 - val_loss: 0.0181 - val_mae: 0.1026
Epoch 3/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0062 - mae: 0.0597 - val_loss: 0.0147 - val_mae: 0.0931
Epoch 4/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0046 - mae: 0.0517 - val_loss: 0.0123 - val_mae: 0.0867
Epoch 5/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0037 - mae: 0.0462 - val_loss: 0.0130 - val_mae: 0.0915
Epoch 6/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0031 - mae: 0.0428 - val_loss: 0.0104 - val_mae: 0.0805
Epoch 7/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.0026 - mae: 0.0394 - val_loss: 0.0099 - val_mae: 0.0778
Epoch 8/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - loss: 0.0022 - mae: 0.0355 - val_loss: 0.0092 - val_mae: 0.0743
Epoch 9/200
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - lo

Model A, the MLP model seems to be performing better with a MAE of 0.0196 meaning the malaria prevalence predicitons were around 1.96% off which is about 98% accuracy.

Hyperparameter tuning would be done on this model to improve the accuracy score.

In [28]:
!pip install -q keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.3 MB/s eta 0:00:00


In [29]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from keras_tuner import HyperModel, RandomSearch

# Define the MLP HyperModel
class MLPHyperModel(HyperModel):
    def __init__(self, input_dim):
        self.input_dim = input_dim

    def build(self, hp):
        inp = layers.Input(shape=(self.input_dim,))

        # Tune number of hidden layers
        x = inp
        for i in range(hp.Int('num_layers', 2, 5)):
            x = layers.Dense(
                units=hp.Int(f'units_{i}', 64, 512, step=64),
                activation=hp.Choice('activation', ['relu', 'elu', 'leaky_relu'])
            )(x)
            x = layers.BatchNormalization()(x)
            x = layers.Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1))(x)

        out = layers.Dense(1, activation='linear')(x)

        model = models.Model(inputs=inp, outputs=out)

        optimizer = tf.keras.optimizers.Adam(
            learning_rate=hp.Float('lr', 1e-4, 1e-2, sampling='log')
        )

        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
        return model

# Instantiate tuner
tuner = RandomSearch(
    MLPHyperModel(X_train.shape[1]),
    objective='val_mae',
    max_trials=20,  # you can increase this for deeper search
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='malaria_mlp_tuning'
)

# Early stopping
es = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Run search
tuner.search(X_train, y_train, validation_split=0.1, epochs=100, batch_size=128, callbacks=[es])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]

# Evaluate
loss, mae = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Best tuned model MAE: {mae:.6f}")

Trial 20 Complete [00h 00m 22s]
val_mae: 0.02020302414894104

Best val_mae So Far: 0.011449514888226986
Total elapsed time: 00h 16m 19s
Best tuned model MAE: 0.046323


### Retrain best model on entire training data

In [30]:
# Build the best version using the tuned hyperparameters
final_model = MLPHyperModel(X_train.shape[1]).build(best_hp)

# Train on full training data
final_model.fit(X_train, y_train, epochs=150, batch_size=128, validation_split=0.1, callbacks=[es])

# Evaluate on test data
loss, mae = final_model.evaluate(X_test, y_test, verbose=0)
print(f"Final model Test MAE: {mae:.6f}")

Epoch 1/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 2.2103 - mae: 1.0541 - val_loss: 0.0158 - val_mae: 0.1059
Epoch 2/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.2967 - mae: 0.3905 - val_loss: 0.0094 - val_mae: 0.0790
Epoch 3/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.0736 - mae: 0.1979 - val_loss: 0.0076 - val_mae: 0.0703
Epoch 4/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0210 - mae: 0.1071 - val_loss: 0.0052 - val_mae: 0.0581
Epoch 5/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.0081 - mae: 0.0674 - val_loss: 0.0039 - val_mae: 0.0504
Epoch 6/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0051 - mae: 0.0525 - val_loss: 0.0028 - val_mae: 0.0430
Epoch 7/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0041 - mae: 0.0480 - val_loss: 0.0012 - val_mae: 0.0273
Epoch 8/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0039 - mae: 0.0467 - val_loss: 0.0012 - val_mae: 0.0282
Epoch 9/150
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 

### Train on entire train and test combined, then export to pickle format

In [31]:
import pickle
import numpy as np

# Combine train and test
X_full = np.vstack([X_train, X_test])
y_full = np.concatenate([y_train, y_test])

# Retrain on full dataset
final_model.fit(X_full, y_full, epochs=150, batch_size=128, callbacks=[es])

# Save to pickle
with open('malaria_mlp_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)

print("Final tuned model saved as malaria_mlp_model.pkl")

Epoch 1/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0028 - mae: 0.0405
Epoch 2/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0026 - mae: 0.0386
Epoch 3/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.0026 - mae: 0.0387
Epoch 4/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0027 - mae: 0.0396
Epoch 5/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0024 - mae: 0.0374
Epoch 6/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0026 - mae: 0.0380
Epoch 7/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0028 - mae: 0.0394
Epoch 8/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0028 - mae: 0.0398
Epoch 9/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0027 - mae: 0.0390
Epoch 10/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0027 - mae: 0.0388
Epoch 11/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0024 - mae: 0.0374
Epoch 12/150
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0026 - mae: 0.0386
Epoch 13/150
76/76 ━━━━━━━━━━━━━━━

In [4]:
!pip install --force-reinstall --no-cache-dir numpy==1.26.4 pmdarima==2.0.4 statsmodels==0.14.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 241.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 178.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 191.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 143.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 160.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 184.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 126.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 162.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 190.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 184.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Load dataset
site_year = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/PHC DataThon Project/model_ready_data.csv")

# Aggregate by year and sum the prevalence
# This will group all rows by 'year' and sum the 'Malaria_Prevalence' values.
df = (
    site_year.groupby('year', as_index=False)['Malaria_Prevalence']
    .sum()
    .rename(columns={'Malaria_Prevalence': 'prevalence'})
)

# Set 'year' as the index for time series modeling
df.set_index('year', inplace=True)

print("Aggregated yearly prevalence data:")
print(df.head())

# Fit SARIMAX model
model = SARIMAX(df['prevalence'], order=(2, 1, 2), seasonal_order=(0, 0, 0, 0))
model_fit = model.fit(disp=False)

# Forecast for the next 8 years (e.g., 2021–2028)
forecast = model_fit.forecast(steps=8)
forecast.index = range(df.index.max() + 1, df.index.max() + 1 + len(forecast))

# Display results
print("\nForecasted Malaria Prevalence (next 8 years):")
print(forecast)

Aggregated yearly prevalence data:
      prevalence
year            
2000  270.549884
2001  265.625461
2002  260.697804
2003  255.898833
2004  251.360471

Forecasted Malaria Prevalence (next 8 years):
2021    167.537619
2022    180.886167
2023    192.626584
2024    202.304605
2025    209.954921
2026    215.825660
2027    220.231377
2028    223.480066
Name: predicted_mean, dtype: float64


/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index

In [17]:
site_year

,year,DHSID,GPS_Dataset,DHSCC,DHSCLUST,SurveyID,All_Population_Count,Aridity,Day_Land_Surface_Temp,Diurnal_Temperature_Range,...,Night_Land_Surface_Temp,PET,Precipitation,Rainfall,U5_Population,UN_Population_Count,UN_Population_Density,Wet_Days,prev_lag1,prev_roll3
0,2000,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,-0.367737,0.026209,-0.274948,0.163110,...,0.203334,-0.204394,0.295579,-0.016443,-0.399418,-0.307618,-0.309646,-0.283441,0.550748,0.483340
1,2001,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,-0.345745,-0.039198,-0.318601,0.038136,...,0.398168,-0.241181,0.159248,-0.155641,-0.358710,-0.306348,-0.308412,-0.308821,0.550748,0.551840
2,2002,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,-0.324294,-0.096300,-0.345079,-0.071418,...,0.569708,-0.265433,0.046336,-0.271467,-0.321199,-0.305031,-0.307131,-0.325012,0.689102,0.606624
3,2003,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,-0.303408,-0.141517,-0.358676,-0.158458,...,0.706462,-0.279654,-0.039207,-0.359579,-0.287864,-0.303669,-0.305806,-0.334052,0.785902,0.704526
4,2004,NG202100000001,NGGE81FL,NG,1.0,NG2021MIS,-0.283110,-0.171272,-0.363685,-0.215885,...,0.796941,-0.286349,-0.093430,-0.415631,-0.259682,-0.302263,-0.304439,-0.337981,0.847357,0.767428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11902,2016,NG202100000568,NGGE81FL,NG,568.0,NG2021MIS,-0.324492,-0.305798,-0.725163,-0.666587,...,-0.095371,-0.633027,-0.552449,-0.334321,-0.363176,-0.102485,-0.110143,0.208983,0.311119,0.308604
11903,2017,NG202100000568,NGGE81FL,NG,568.0,NG2021MIS,-0.316862,-0.298205,-0.628397,-0.666765,...,-0.059578,-0.639048,-0.544199,-0.381229,-0.355259,-0.095561,-0.103409,0.127644,0.362029,0.330867
11904,2018,NG202100000568,NGGE81FL,NG,568.0,NG2021MIS,-0.307301,-0.277596,-0.531711,-0.667250,...,-0.024530,-0.650637,-0.521807,-0.424477,-0.347211,-0.088534,-0.096575,0.052460,0.517157,0.485665
11905,2019,NG202100000568,NGGE81FL,NG,568.0,NG2021MIS,-0.296263,-0.237462,-0.435106,-0.668195,...,0.009746,-0.669067,-0.478202,-0.463528,-0.339033,-0.081408,-0.089644,-0.015701,0.780102,0.747223


Forecast Malaria Prevalence Till 2028